# Camera Calibration

In [363]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        #img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

In [364]:
#cv2.destroyAllWindows()

In [366]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [38]:
print (ret)
print (mtx)
print (dist)
print (rvecs)
print (tvecs)

0.15536906910078757
[[534.07088364   0.         341.53407554]
 [  0.         534.11914595 232.94565259]
 [  0.           0.           1.        ]]
[[-2.92971637e-01  1.07706962e-01  1.31038376e-03 -3.11018780e-05
   4.34798110e-02]]
[array([[-0.45883216],
       [-0.08848877],
       [-1.33510786]]), array([[-0.17288944],
       [-0.46764681],
       [ 1.34745198]]), array([[-0.43239599],
       [ 0.25603401],
       [-3.08832021]]), array([[-0.2645143 ],
       [-0.39360849],
       [-2.74787379]]), array([[ 0.41531697],
       [ 0.65664497],
       [-1.3373494 ]]), array([[-0.35367631],
       [-0.24363035],
       [-1.56874295]]), array([[-0.29979221],
       [ 0.39216377],
       [-1.4348239 ]]), array([[-0.32034625],
       [ 0.1597993 ],
       [-1.24149595]]), array([[ 0.30697385],
       [ 0.5038552 ],
       [-1.82824733]]), array([[-0.45993978],
       [-0.3142018 ],
       [-1.76122223]]), array([[-0.37843358],
       [-0.18064237],
       [-3.11615996]])]
[array([[-3.279164

In [365]:
img = cv2.imread('left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [367]:
# undistort (first method)
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)

True

In [6]:
# undistort (second method)
mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)

True

In [368]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print ("total error: ", mean_error/len(objpoints))

total error:  0.023686000375385673


# Stamp Perspective Wraping

In [212]:
import glob
from skimage import exposure
import matplotlib.pyplot as plt
import numpy as np
import argparse
import imutils
import cv2
from scipy.spatial import distance
%matplotlib inline

In [369]:
images = glob.glob('images/*.jpeg')

In [371]:
# load the query image, compute the ratio of the old height to the new height (will be used in latter plots), 
# clone it, and resize it
image = cv2.imread(images[1])
ratio = image.shape[0] / 300.0
orig = image.copy()
image = imutils.resize(image, height = 300)
 
# convert the image to grayscale, blur it, and find edges
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 11, 17, 17)
edged = cv2.Canny(gray, 30, 200)
cv2.imshow("Stamp", edged)
cv2.waitKey(0)

27

In [372]:
cv2.destroyWindow

<function destroyWindow>

In [373]:
# find contours in the edged image, keep only the largest ones, and initialize stamp contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
stampCnt = None

# loop over contours
for c in cnts:
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.015 * peri, True)
 
	# if our approximated contour has six points, then
	# we can assume that we have found the stamp
	if len(approx) == 6:
		stampCnt = approx
		break

In [374]:
cv2.drawContours(image, [stampCnt], -1, (0, 255, 0), 3)
cv2.imshow("Stamp", image)
cv2.waitKey(0)

27

In [375]:
cv2.destroyWindow

<function destroyWindow>

In [377]:
# now that we have our stamp contour, we need to determine
# the top-left, top-right, bottom-right, and bottom-left
# points so that we can later warp the image -- we'll start
# by reshaping our contour to be our finals and initializing
# our output rectangle in top-left, top-right, bottom-right,
# and bottom-left order
pts = stampCnt.reshape(6, 2)
rect = np.zeros((4, 2), dtype = "float32")
origrect = np.zeros((4, 2), dtype = "float32")
# the top-left point has the smallest sum whereas the
# bottom-right has the largest sum
s = pts.sum(axis = 1)
rect[0] = pts[np.argmin(s)]
rect[2] = pts[np.argmax(s)]
origrect[0] = rect[0]
origrect[2] = rect[2]

# compute the difference between the points -- the top-right
# will have the minumum difference and the bottom-left will
# have the maximum difference
diff = np.diff(pts, axis = 1)
rect[1] = pts[np.argmin(diff)]
rect[3] = pts[np.argmax(diff)]
origrect[1] = rect[1]
origrect[3] = rect[3]

# multiply the rectangle by the original ratio
rect *= ratio

In [326]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    return nodes[closest_index]

In [340]:
nodeslist = []
for n in pts:
    if n[0] != origrect[0][0] and n[1] != origrect[0][1]:
        nodeslist.append(n)

p1 = closest_node(origrect[0], nodeslist)

nodeslist = []
for n in pts:
    if n[0] != origrect[1][0] and n[1] != origrect[1][1]:
        nodeslist.append(n)
p2 = closest_node(origrect[1], nodeslist)

nodeslist = []
for n in pts:
    if n[0] != origrect[2][0] and n[1] != origrect[2][1]:
        nodeslist.append(n)
p3 = closest_node(origrect[2], nodeslist)

nodeslist = []
for n in pts:
    if n[0] != origrect[3][0] and n[1] != origrect[3][1]:
        nodeslist.append(n)
p4 = closest_node(origrect[3], nodeslist)

In [354]:
if p1 not in origrect and p4 not in origrect:
    rotation = 0

elif p2 not in origrect and p1 not in origrect:
    rotation = 90

elif p3 not in origrect and p2 not in origrect:
    rotation = 180

elif p4 not in origrect and p3 not in origrect:
    rotation = -90

In [358]:
# now that we have our rectangle of points, let's compute
# the width of our new image
(tl, tr, br, bl) = rect
widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
 
# and now for the height of our new image
heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
 
# take the maximum of the width and height values to reach
# our final dimensions
maxWidth = max(int(widthA), int(widthB))
maxHeight = max(int(heightA), int(heightB))
 
# construct our destination points which will be used to
# map the stamp to a top-down, "birds eye" view
dst = np.array([
    [0, 0],
    [maxWidth - 1, 0],
    [maxWidth - 1, maxHeight - 1],
    [0, maxHeight - 1]], dtype = "float32")
 
# calculate the perspective transform matrix and warp
# the perspective to grab the stamp
M = cv2.getPerspectiveTransform(rect, dst)
warp = cv2.warpPerspective(orig, M, (maxWidth, maxHeight))

In [360]:
def rotate(image, angle, center=None, scale=1.0):

    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated

In [380]:
rotated = rotate(warp, rotation)
cv2.imshow("stamp", rotated)
cv2.waitKey(0)

27

In [381]:
cv2.destroyWindow

<function destroyWindow>

In [378]:
cv2.imshow("stamp", warp)
cv2.waitKey(0)

27